# Acquire New Data

## Extension Plan Motivation and Focus:

The following paragraphs are pulled directly from the 'Part 2 Extension Plan.pdf' file I previously wrote:

The objective for my project extension is to dig deeper into the wildfire smoke analysis I did in Part 1 regarding Richland, Washington. I want to study how the smoke from these wildfires may impact the health of the residents of Richland, WA. This is an important topic for the city leaders to consider, as the findings, if valid, would guide their decision-making in how to allocate funds to promote the well-being of their citizens. For example, if the rate of hospitalization for respiratory illnesses is increasing, perhaps more hospital staff, beds, etc. should be provided. Additionally, the people of Richland would be very interested in learning how wildfires might impact their health in future years. Thus, there is a societal benefit to studying this topic further.

More specifically, for my extended analysis, I wish to answer the question “How will wildfire smoke impact the number of hospitalizations for respiratory issues in Richland, WA in the coming years?” To answer this question, I plan on acquiring population data and healthcare data to supplement the wildfire data I gathered in Part 1. Then, I will find the correlation between the smoke and hospitalizations in the county that Richland is located in. I will also use a simple model (probably a regression model) to predict future hospitalization rates for the future years. Lastly, I will document my findings, visualize the results, and provide a discussion on some of the dominant limitations and assumptions of my analysis.

**Author: Logan O'Brien**

In [1]:
#imports
import pandas as pd

## Step 1: Retrieve Population Data

The first data I want to gather for the Project Part 2 extended analysis is U.S. Census population data [1]. The first thing to consider before retrieving this data is what years do we need? Since I plan on building a model that incorporates, among other things, population, hospitalization rate, and smoke estimate (via AQI measurements), the longevity of the data will be reduced to the common denominator of the dataset with the smallest range of data. Well, I know from my exploration when writing my extension plan document and from my work in Project Part 1, that the healthcare data I will get later appears to be the limting factor and only provides data from 2011 to 2020 (with at least one year in that range missing as well). Thus, I will gather population estimates from around 2010 through 2020.

The U.S. Census website provides a dataset for county population estimates for 2010 - 2019 here: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html [2] and for 2020 - 2022 here: https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-total.html [3]. This data is publicaly available and I called to confirm that I could use their data as long as I cite it. 

Let's load and merge the data into a single dataset

In [2]:
# load the county population estimate data for Washington for 2010 - 2019
df_2010_19 = pd.read_excel('https://www2.census.gov/programs-surveys/popest/tables/2010-2019/counties/totals/co-est2019-annres-53.xlsx')

# load the county population estimate data for Washington for 2020 - 2022
df_2020_22 = pd.read_excel('https://www2.census.gov/programs-surveys/popest/tables/2020-2022/counties/totals/co-est2022-pop-53.xlsx')

In [3]:
df_2010_19

,table with row headers in column A and column headers in rows 3 through 4 (leading dots indicate sub-parts),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geographic Area,2010-04-01 00:00:00,NaN,Population Estimate (as of July 1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Census,Estimates Base,2010,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
3,Washington,6724540,6724540,6742830,6826627.0,6897058.0,6963985.0,7054655.0,7163657.0,7294771.0,7423362.0,7523869.0,7614893.0
4,".Adams County, Washington",18728,18731,18790,18877.0,18944.0,19098.0,19177.0,19233.0,19369.0,19651.0,19736.0,19983.0
5,".Asotin County, Washington",21623,21623,21725,21971.0,21909.0,22129.0,22190.0,22113.0,22286.0,22509.0,22616.0,22582.0
6,".Benton County, Washington",175177,175168,176465,180436.0,182373.0,184318.0,186489.0,190218.0,193494.0,198200.0,201286.0,204390.0
7,".Chelan County, Washington",72453,72460,72750,73214.0,73472.0,73723.0,74121.0,75041.0,75855.0,76298.0,76752.0,77200.0
8,".Clallam County, Washington",71404,71396,71503,71762.0,71766.0,72046.0,72467.0,73202.0,74240.0,75637.0,76551.0,77331.0
9,".Clark County, Washington",425363,425360,426704,432283.0,436361.0,441341.0,448202.0,456939.0,465272.0,474381.0,481427.0,488241.0


This dataframe is messy because the data comes from a table with several rows of merged cells and headers. We only need the data for Benton County. Let's start by getting the column headers.

In [4]:
cols = list(df_2010_19.iloc[2, 3:])
cols

[2010, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0]

Note that the values above are of mixed type. Let's fix this.

In [6]:
# convert numbers to integers to drop decimals
# Source: I learned how to do list comprehensions from:
# https://www.geeksforgeeks.org/python-program-to-convert-list-of-integer-to-list-of-string/#
years_ints = [int(x) for x in cols]

# convert elements to string
years_str = [str(elt) for elt in years_ints]
years_str

['2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

Now, we can finish extracting the Benton County data for 2010 - 2019.

In [7]:
# extract the data we need
data = list(df_2010_19.iloc[6, 3:])

# I referenced this video to learn that I need to put the data variable below in brackets:
# Source: https://www.youtube.com/watch?v=Nuf8edUbgC8
df_Benton_2010_19 = pd.DataFrame([data], columns=years_str)
df_Benton_2010_19

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,176465,180436.0,182373.0,184318.0,186489.0,190218.0,193494.0,198200.0,201286.0,204390.0


Now, let's similarly retrieve the Benton County data from the 2020 - 2022 dataset we loaded previously.

In [8]:
df_2020_22

,table with row headers in column A and column headers in rows 3 through 4 (leading dots indicate sub-parts),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
1,Geographic Area,"April 1, 2020 Estimates Base",Population Estimate (as of July 1),NaN,NaN
2,NaN,NaN,2020,2021.0,2022.0
3,Washington,7705247,7724031,7740745.0,7785786.0
4,".Adams County, Washington",20612,20608,20640.0,20961.0
5,".Asotin County, Washington",22288,22325,22468.0,22508.0
6,".Benton County, Washington",206875,207413,210585.0,212791.0
7,".Chelan County, Washington",79078,79228,79775.0,79926.0
8,".Clallam County, Washington",77160,77357,78442.0,77805.0
9,".Clark County, Washington",503309,505301,512588.0,516779.0


Once again, we must extract and reformat some of the data using the same process we did for the other file.

In [12]:
cols = list(df_2020_22.iloc[2, 2:])
cols

# convert numbers to integers to drop decimals
# Source: I learned how to do list comprehensions from:
# https://www.geeksforgeeks.org/python-program-to-convert-list-of-integer-to-list-of-string/#
years_ints = [int(x) for x in cols]

# convert elements to string
years_str = [str(elt) for elt in years_ints]

# extract the data we need
data = list(df_2020_22.iloc[6, 2:])

# I referenced this video to learn that I need to put the data variable below in brackets:
# Source: https://www.youtube.com/watch?v=Nuf8edUbgC8
df_Benton_2020_22 = pd.DataFrame([data], columns=years_str)
df_Benton_2020_22

,2020,2021,2022
0,207413,210585.0,212791.0


Now, let's merge the two dataframes into a single dataframe containing population estimates for 2010 - 2022.

In [13]:
# I consulted this source to learn how to add two dataframes together horizantally
# Source: https://stackoverflow.com/questions/44723377/pandas-combining-two-dataframes-horizontally
df_Benton_pop_est = pd.concat([df_Benton_2010_19, df_Benton_2020_22], axis=1)
df_Benton_pop_est

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,176465,180436.0,182373.0,184318.0,186489.0,190218.0,193494.0,198200.0,201286.0,204390.0,207413,210585.0,212791.0


## Step 2: Retrieve Healthcare Data

After gathering the population data, we now move on to gleaning the healthcare data that we wish to use. For this, we are going to use a data analysis and visualization tool provided by the Agency for Healthcare Research and Quality (AHRQ) [4] to export the data we need. This tool is part of a project called the Healthcare Cost and Utilization Project (HCUP) [5]. Note, in order to use this tool, users are required to accept the terms of use, which in essence stipulate that the user must make no efforts to identify individuals or establishments in the data.

The data we will download from the tool [5] comes from the "Community Inpatient Statistics" tab which allows us to download the number of discharges per county for a selected state, time period, and year. Additionally, by clicking the "Diagnoses & Conditions" subtab, we can further filter by a specific diagnosis. Before pulling any data, we must first determine which diagnoses we want data for. 

In my survey of some of the current literature on the health impacts of wildfire smoke, in preperation for drafting my Part 2 extension plan, I learned that several of the same respiratory conditions repeatedly appeared. These respiratory issues are: pneumonia, chronic obstructive pulmonary disease and allied conditions, pneumonitis, asthma, (acute) bronchitis, and wheezing, respiratory tract infections,  [6 - 8].

What HCUP provides:
Batch 1:
- RSP002: Pneumonia
- RSP005: Acute bronchitis
- RSP008: COPD and bronchiectasis?
- RSP009: Asthma
- RSP010: Aspiration pneumonitis?

Batch 2 (CCS):
- 122: Pneumonia
- 125: Acute bronchitis
- 127: COPD and bronchiectasis
- 128: Asthma
- 129: aspiration pneumonitis; food/vomitus???


## References:
- [1]. https://www.census.gov/
- [2]. Annual Estimates of the Resident Population for Counties in Washington: April 1, 2010 to July 1, 2019 (CO-EST2019-ANNRES-53). Source: U.S. Census Bureau, Population Division. Release Date: March 2020.
- [3]. Annual Estimates of the Resident Population for Counties in Washington: April 1, 2020 to July 1, 2022 (CO-EST2022-POP-53). Source: U.S. Census Bureau, Population Division. Release Date: March 2023. 
- [4]. https://www.ahrq.gov/
- [5]. https://datatools.ahrq.gov/hcupnet/





- [6]. DeFlorio-Barker S, Crooks J, Reyes J, Rappold AG. Cardiopulmonary effects of fine particulate matter exposure among older adults, during wildfire and non-wildfire periods, in the United States 2008-2010. Environ Health Perspect 2019;127(3):37006. doi: 10.1289/ehp3860.
- [7]. Liu JC, Wilson A, Mickley LJ, Dominici F, Ebisu K, Wang Y, et al.. 2017. Wildfire-specific fine particulate matter and risk of hospital admissions in urban and rural counties. Epidemiology 28(1):77–85, PMID: 27648592, 10.1097/EDE.0000000000000556. [PMC free article] [PubMed] [CrossRef] [Google Scholar]
- [8]. Rappold AG, Stone SL, Cascio WE, Neas LM, Kilaru VJ, Carraway MS, et al.. 2011. Peat bog wildfire smoke exposure in rural North Carolina is associated with cardiopulmonary emergency department visits assessed through syndromic surveillance. Environ Health Perspect 119(10):1415–1420, PMID: 21705297, 10.1289/ehp.1003206. [PMC free article] [PubMed] [CrossRef] [Google Scholar]
- [9]. Delfino RJ, Brummel S, Wu J, Stern H, Ostro B, Lipsett M, et al.. 2009. The relationship of respiratory and cardiovascular hospital admissions to the southern California wildfires of 2003. Occup Environ Med 66(3):189–197, PMID: 19017694, 10.1136/oem.2008.041376. [PMC free article] [PubMed] [CrossRef] [Google Scholar]
- [10]. Moore D, Copes R, Fisk R, Joy R, Chan K, Brauer M. 2006. Population health effects of air quality changes due to forest fires in British Columbia in 2003: estimates from physician-visit billing data. Can J Public Health 97(2):105–108, PMID: 16619995. [PMC free article] [PubMed] [Google Scholar]
    - https://link.springer.com/article/10.1007/BF03405325
- [11]. Liu JC, Pereira G, Uhl SA, Bravo MA, Bell ML. 2015. A systematic review of the physical health impacts from non-occupational exposure to wildfire smoke. Environ Res 136:120–132, PMID: 25460628, 10.1016/j.envres.2014.10.015. [PMC free article] [PubMed] [CrossRef] [Google Scholar]
- [12]. Ignotti E, Valente JG, Longo KM, Freitas SR, Hacon Sde S, Netto PA. Impact on human health of particulate matter emitted from burnings in the Brazilian Amazon region. Rev Saude Publica. 2010;44:121–130. [PubMed] [Google Scholar]
    - https://www.scielo.br/j/rsp/a/bp9BffF785sJmcC6hqX366d/?lang=en
- [13]. de Mendonca MJ, et al. Estimation of damage to human health due to forest burning in the Amazon. J Popul Econ. 2006;19:593–610. [Google Scholar]


-	[1]. DeFlorio-Barker S, Crooks J, Reyes J, Rappold AG. Cardiopulmonary effects of fine particulate matter exposure among older adults, during wildfire and non-wildfire periods, in the United States 2008-2010. Environ Health Perspect 2019;127(3):37006. doi: 10.1289/ehp3860.
-	[2]. Liu JC, Wilson A, Mickley LJ, Dominici F, Ebisu K, Wang Y, et al.. 2017. Wildfire-specific fine particulate matter and risk of hospital admissions in urban and rural counties. Epidemiology 28(1):77–85, PMID: 27648592, 10.1097/EDE.0000000000000556. [PMC free article] [PubMed] [CrossRef] [Google Scholar]
-	[3]. Rappold AG, Stone SL, Cascio WE, Neas LM, Kilaru VJ, Carraway MS, et al.. 2011. Peat bog wildfire smoke exposure in rural North Carolina is associated with cardiopulmonary emergency department visits assessed through syndromic surveillance. Environ Health Perspect 119(10):1415–1420, PMID: 21705297, 10.1289/ehp.1003206. [PMC free article] [PubMed] [CrossRef] [Google Scholar]
-	[4]. Delfino RJ, Brummel S, Wu J, Stern H, Ostro B, Lipsett M, et al.. 2009. The relationship of respiratory and cardiovascular hospital admissions to the southern California wildfires of 2003. Occup Environ Med 66(3):189–197, PMID: 19017694, 10.1136/oem.2008.041376. [PMC free article] [PubMed] [CrossRef] [Google Scholar]
-	[5]. Moore D, Copes R, Fisk R, Joy R, Chan K, Brauer M. 2006. Population health effects of air quality changes due to forest fires in British Columbia in 2003: estimates from physician-visit billing data. Can J Public Health 97(2):105–108, PMID: 16619995. [PMC free article] [PubMed] [Google Scholar]
o	https://link.springer.com/article/10.1007/BF03405325
-	[6]. Liu JC, Pereira G, Uhl SA, Bravo MA, Bell ML. 2015. A systematic review of the physical health impacts from non-occupational exposure to wildfire smoke. Environ Res 136:120–132, PMID: 25460628, 10.1016/j.envres.2014.10.015. [PMC free article] [PubMed] [CrossRef] [Google Scholar]
o	A good meta analysis
-	[7]. Ignotti E, Valente JG, Longo KM, Freitas SR, Hacon Sde S, Netto PA. Impact on human health of particulate matter emitted from burnings in the Brazilian Amazon region. Rev Saude Publica. 2010;44:121–130. [PubMed] [Google Scholar]
o	https://www.scielo.br/j/rsp/a/bp9BffF785sJmcC6hqX366d/?lang=en
-	[8]. de Mendonca MJ, et al. Estimation of damage to human health due to forest burning in the Amazon. J Popul Econ. 2006;19:593–610. [Google Scholar]
